In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets
import matplotlib.pyplot as plt

from s7_datasets import load_data_S7_1
from models import model_summary
from utils import train_S7,test_S7,plot_acc_loss_S7

c:\Users\Aai\anaconda3\envs\pytorch\Lib\site-packages\torchvision\io\image.py:13: UserWarning: Failed to load image Python extension: '[WinError 127] The specified procedure could not be found'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(
c:\Users\Aai\anaconda3\envs\pytorch\Lib\site-packages\transformers\utils\generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


In [2]:
use_cuda = torch.cuda.is_available()  # Check if CUDA (GPU support) is available
device = torch.device("cuda" if use_cuda else "cpu")  # Select GPU if available, otherwise fall back to CPU
device  # Display the selected device

device(type='cpu')

In [3]:
train_loader , test_loader = load_data_S7_1()

In [4]:
class S7_Model_2(nn.Module):
    def __init__(self):
        super(S7_Model_2, self).__init__()
        self.conv1 = nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=32, kernel_size=7, padding=3, bias=False), 
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.Dropout(0.25)

            )
        
        self.trans_block1 = nn.Sequential(
            nn.MaxPool2d(2, 2), 
            nn.Conv2d(32, 16, 1), 
            nn.BatchNorm2d(16),
            nn.ReLU()
        )
        self.conv2 = nn.Sequential(
            nn.Conv2d(in_channels=16, out_channels=8, kernel_size=5, padding=2),  
            nn.BatchNorm2d(8),
            nn.ReLU(),
            nn.Dropout(0.25)
            
        )
        self.conv3 = nn.Sequential(
            nn.Conv2d(in_channels=8, out_channels=4, kernel_size=3, padding=1),  
            nn.BatchNorm2d(4),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),
            nn.Dropout(0.25)  
        )
        self.trans_block2 = nn.Sequential(
            nn.MaxPool2d(2, 2), 
            nn.Conv2d(4, 32, 1), 
            nn.BatchNorm2d(32),
            nn.ReLU()
        )
        
        self.conv4 = nn.Sequential(
            nn.Conv2d(32, 16, kernel_size=3, padding=1),
            nn.ReLU(),     
        )
        self.conv5 = nn.Sequential(
            nn.Conv2d(16, 10, kernel_size=1),
            nn.ReLU(),
        )
        

        
        
        # self.conv1x1_2 = nn.Sequential(
        #     nn.Conv2d(in_channels=8, out_channels=2, kernel_size=3, padding=1), 
        #     nn.BatchNorm2d(2),
        #     nn.ReLU(),
        # )
    def forward(self, x):
            x = self.conv1(x)
            x = self.trans_block1(x)
            x = self.conv2(x)
            x = self.conv3(x)
            x = self.trans_block2(x)
            x = self.conv4(x)
            x = self.conv5(x)

            x = x.view(x.size(0), -1)  
            x = F.log_softmax(x, dim=1)
            return x

In [5]:
model = S7_Model_2()
model_summary(model,input_size=(1,28,28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 32, 28, 28]           1,568
       BatchNorm2d-2           [-1, 32, 28, 28]              64
              ReLU-3           [-1, 32, 28, 28]               0
           Dropout-4           [-1, 32, 28, 28]               0
         MaxPool2d-5           [-1, 32, 14, 14]               0
            Conv2d-6           [-1, 16, 14, 14]             528
       BatchNorm2d-7           [-1, 16, 14, 14]              32
              ReLU-8           [-1, 16, 14, 14]               0
            Conv2d-9            [-1, 8, 14, 14]           3,208
      BatchNorm2d-10            [-1, 8, 14, 14]              16
             ReLU-11            [-1, 8, 14, 14]               0
          Dropout-12            [-1, 8, 14, 14]               0
           Conv2d-13            [-1, 4, 14, 14]             292
      BatchNorm2d-14            [-1, 4,

In [6]:
model = S7_Model_2().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01,momentum=0.9)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=15, gamma=0.1, verbose=True)
# New Line
criterion = F.nll_loss
num_epochs = 15




for epoch in range(1, num_epochs+1):
  print(f'Epoch {epoch}')
  train_S7(model, device, train_loader, optimizer,criterion)
  test_S7(model, device, test_loader, criterion)
  scheduler.step()

c:\Users\Aai\anaconda3\envs\pytorch\Lib\site-packages\torch\optim\lr_scheduler.py:28: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn("The verbose parameter is deprecated. Please use get_last_lr() "


Epoch 1


Train: Loss=0.2448 Batch_id=937 Accuracy=81.19: 100%|██████████| 938/938 [01:12<00:00, 13.02it/s]


Test set: Average loss: 0.0058, Accuracy: 9060/10000 (90.60%)

Epoch 2


Train: Loss=0.4516 Batch_id=937 Accuracy=91.92: 100%|██████████| 938/938 [01:09<00:00, 13.46it/s]


...

Test set: Average loss: 0.0031, Accuracy: 9435/10000 (94.35%)

Epoch 14


Train: Loss=0.1352 Batch_id=937 Accuracy=96.34: 100%|██████████| 938/938 [00:37<00:00, 25.28it/s]


Test set: Average loss: 0.0024, Accuracy: 9562/10000 (95.62%)

Epoch 15


Train: Loss=0.0420 Batch_id=937 Accuracy=96.38: 100%|██████████| 938/938 [00:36<00:00, 26.04it/s]


Test set: Average loss: 0.0017, Accuracy: 9714/10000 (97.14%)

